# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env 
# (e.g., model weights, data directories) resolve correctly
%cd ../

In [ ]:
import sys
sys.path.append('.')

import pickle
from io import BytesIO
import requests
from app.s3 import files_service

In [ ]:
INPUT_VIDEO_PATH = "test.mp4"
VIDEO_KEY = "test2/video"
OUTPUT_VIDEO_PATH = "2.downloaded_test.mp4"

PICKLE_KEY = "test2/pickle"
PICKLE_DATA = {"hello": "world!"}

In [ ]:
# Start MinIO — required by code that interacts with object storage
!pixi run just minio-up

In [ ]:
# # After finishing your work, shut down MinIO to free resources
# !pixi run just minio-down

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'test.png' instead of 'assets/test.png')
%cd playground-testing/assets/

# **Test `FileService`**

## **I. Test `create_bucket()` method**

In [ ]:
files_service.create_bucket()

## **II. Test `upload_fileobj()` method**

In [ ]:
def test_upload_video(storage_key: str, file_path: str) -> None:

    with open(file_path, "rb") as f:
        files_service.upload_fileobj(storage_key=storage_key, fileobj=f)
    
    print(f"Successfully uploaded video file '{file_path}' to storage key '{storage_key}'.")

In [ ]:
test_upload_video(storage_key = VIDEO_KEY, file_path = INPUT_VIDEO_PATH)

In [ ]:
def test_upload_pickle(storage_key: str, data) -> None:

    pickle_bytes = pickle.dumps(data)
    fileobj = BytesIO(pickle_bytes)
    files_service.upload_fileobj(storage_key=storage_key, fileobj=fileobj)
    print(f"Successfully uploaded pickle file to storage key '{storage_key}'.")

In [ ]:
test_upload_pickle(storage_key = PICKLE_KEY, data = PICKLE_DATA)

## **III. Test `head_object()` method**

In [ ]:
def test_head_object(storage_key: str) -> None:

    # Step 1: Attempt to retrieve object metadata
    try:
        metadata = files_service.head_object(storage_key)
    except Exception as e:
        print(f"Object '{storage_key}' does not exist or an error occurred: {e}")
        return

    # Step 2: Confirm success and show basic info
    content_length = metadata.get("ContentLength", "unknown")
    last_modified = metadata.get("LastModified", "unknown")
    print(f"Object '{storage_key}' exists. Size: {content_length} bytes, Last modified: {last_modified}")

In [ ]:
test_head_object(storage_key = VIDEO_KEY)

In [ ]:
test_head_object(storage_key = PICKLE_KEY)

## **IV. Test `generate_presigned_url()` method (with video download and local verification)**

In [ ]:
def test_download_video(storage_key: str, output_filename: str) -> None:
    
    # Step 1: Generate presigned URL
    presigned_url = files_service.generate_presigned_url(storage_key, expires=300)

    # Step 2: Stream-download the video
    response = requests.get(presigned_url, stream=True)
    response.raise_for_status()  # raises HTTPError for bad status

    # Step 3: Save to disk
    with open(output_filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Successfully downloaded video file '{storage_key}' from MinIO and saved to '{output_filename}'.")

In [ ]:
test_download_video(storage_key = VIDEO_KEY, output_filename = OUTPUT_VIDEO_PATH)

## **V. Test `download_as_bytes()` method (load pickle)**

In [ ]:
def test_load_pickle(storage_key: str):

    content_bytes = files_service.download_as_bytes(storage_key)
    data = pickle.loads(content_bytes)
    print(f"Successfully loaded pickle object from storage key '{storage_key}'.")
    return data

In [ ]:
test_load_pickle(storage_key = PICKLE_KEY)

## **VI. Test `delete()` method**

In [ ]:
def test_delete_file(storage_key: str) -> None:

    files_service.delete(storage_key=storage_key)
    print(f"File '{storage_key}' was deleted (or did not exist).")

In [ ]:
test_delete_file(storage_key = VIDEO_KEY)

In [ ]:
test_delete_file(storage_key = PICKLE_KEY)